In [ ]:
import pandas as pd

# Читаем и выводим первые 5 строк отзывов из csv файла
data = pd.read_csv('Data/geo-reviews-dataset-2023.csv')
data.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [ ]:
import re
import pandas as pd
import nltk
import stanza
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Data/geo-reviews-dataset-2023.csv', encoding='utf-8')
# Инициализация инструментов
nltk.download('stopwords')
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

# Определение стоп-слов для удаления из текста
stop_words = set(nltk.corpus.stopwords.words('russian'))
additional_stops = {'что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'руб', 'мой', 'твой', 'его', 'её', 'наш', 'ваш', 'их', 'свой', 'еще', 'очень', 'поэтому', 'однако', 'конечно'}
stop_words.update(additional_stops)

# Функция предобработки текста (приведение к нижнему регистру и удаление лишних символов)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    return text

# Применение предобработки
df['aspects'] = df['text'].astype(str).apply(preprocess_text)

def find_top_words_by_rubric(vectorizer):

    result = {
        'rubrics': [],
        'words': [],
        'reviews': [],
        'scores': []
    }

    #Проходимся по рубрикам
    for rubric in df_flattened['rubrics'].unique():
        texts = df_flattened[df_flattened['rubrics'] == rubric]['aspects']
        total_count = texts.shape[0]

        # В анализ возьмём только те рубрики, у которых есть несколько текстов
        if total_count >= 5:
            tfidf_matrix = vectorizer.fit_transform(texts)
        else:
            continue

        result['rubrics'].append(rubric)
        result['reviews'].append(total_count)
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.max(axis=0).toarray().ravel()

        # Возьмём топ-20 слов для каждой рубрики
        top_words_indices = tfidf_scores.argsort()[-20:][::-1]
        top_words = [feature_names[i] for i in top_words_indices]
        result['words'].append(', '.join(top_words))
        top_scores = [str(tfidf_scores[i]) for i in top_words_indices]
        result['scores'].append(', '.join(top_scores))

    return result

# Развернём датасет по рубрикам, так как одна организация может принадлежать к списку рубрик
df['rubrics'] = df['rubrics'].apply(lambda x: x.split(";"))
df_flattened = df.explode('rubrics')

# Инициализируем TF-IDF-векторизатор
aspects_vectorizer = TfidfVectorizer(use_idf = True, max_df = 0.8, min_df = 0.1)

# Создадим датафрейм с результатами анализа
tf_idf_aspects = pd.DataFrame(find_top_words_by_rubric(aspects_vectorizer)).sort_values(by='reviews', ascending=False)

tf_idf_aspects.head()

,rubrics,words,reviews,scores
13,Кафе,"это, что, вкусная, вкусно, все, всегда, всё, д...",58496,"1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."
22,Ресторан,"это, для, как, из, заведение, за, есть, еда, в...",56761,"1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."
25,Гостиница,"это, за, не, на, мы, можно, место, как, из, за...",43133,"1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."
1,Магазин продуктов,"это, что, большой, все, всегда, всё, выбор, дл...",21346,"1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."
46,Супермаркет,"это, много, большой, все, всегда, всё, выбор, ...",19746,"1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1..."


In [ ]:
import pandas as pd
import re
import nltk
import stanza
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
import joblib
from tqdm import tqdm
from scipy.sparse import vstack
import numpy as np

# Загрузка данных из CSV-файла
df = pd.read_csv('Data/geo-reviews-dataset-2023.csv', encoding='utf-8')
df = df.loc[df['rubrics'] == 'Гостиница']  # Оставляем только гостиницы

# Инициализация инструментов для обработки текста
nltk.download('stopwords')
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

# Определение стоп-слов для удаления из текста
stop_words = set(nltk.corpus.stopwords.words('russian'))
additional_stops = {'что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'руб', 'мой', 'твой', 'его', 'её', 'наш', 'ваш', 'их', 'свой', 'еще', 'очень', 'поэтому', 'однако', 'конечно'}
stop_words.update(additional_stops)

# Функция предобработки текста (приведение к нижнему регистру и удаление лишних символов)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    return text

# Применение предобработки к отзывам
df['clean_text'] = df['text'].astype(str).apply(preprocess_text)

# Группировка отзывов по отелям и объединение их в один текст
df_grouped = df.groupby('name_ru')['clean_text'].apply(lambda x: ' '.join(x)).reset_index()
df_grouped['rating'] = df.groupby('name_ru')['rating'].mean().reset_index(drop=True)  # Средний рейтинг отеля

# Сохранение сгруппированных данных в CSV
df_grouped.to_csv('Data/grouped_hotels.csv', index=False, encoding='utf-8')

# Векторизация текстов отзывов с использованием TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=list(stop_words))
X = vectorizer.fit_transform(df_grouped['clean_text'])
y = df_grouped['rating'].values

# Инициализация модели случайного леса для регрессии
model = RandomForestRegressor(n_estimators=100, random_state=42, warm_start=True)
batch_size = 100  # Размер батча для обучения

# Обучение модели по батчам
for i in tqdm(range(0, X.shape[0], batch_size), desc="Обучение модели по батчам"):
    X_batch = X[i:i+batch_size]
    y_batch = y[i:i+batch_size]
    
    if i == 0:
        model.fit(X_batch, y_batch)  # Первоначальное обучение модели
    else:
        model.n_estimators += 10  # Увеличение количества деревьев в лесу
        model.fit(X_batch, y_batch)

# Сохранение обученной модели и векторайзера в файлы
joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Компуктер\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2025-03-23 18:43:20 INFO: Downloaded file to C:\Users\Компуктер\stanza_resources\resources.json
2025-03-23 18:43:20 INFO: Downloading default packages for language: ru (Russian) ...
2025-03-23 18:43:21 INFO: File exists: C:\Users\Компуктер\stanza_resources\ru\default.zip
2025-03-23 18:43:25 INFO: Finished downloading models and saved to C:\Users\Компуктер\stanza_resources
2025-03-23 18:43:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-23 18:43:26 INFO: Downloaded file to C:\Users\Компуктер\stanza_resources\resources.json
2025-03-23 18:43:26 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| lemma     | syntagrus_nocharlm |

2025-03-23 18:43:26 INFO: Using device: cuda
2025-03-23 18:43:26 INFO: Loading: tokenize
2025-03-23 18:43:26 INFO: Loading: lemma
2025-03-23 18:43:29 INFO: Done loading processors!
Обучение модели по батчам: 100%|██████████| 106/106 [00:25<00:00,  4.16it/s]


['vectorizer.pkl']

In [4]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка модели и данных
model = joblib.load("model.pkl")
vectorizer = joblib.load("vectorizer.pkl")
df_grouped = pd.read_csv("Data/grouped_hotels.csv")

# Функция поиска отелей
def recommend_hotels():
    user_input = entry.get()
    
    if not user_input.strip():
        messagebox.showwarning("Ошибка", "Введите предпочтения перед поиском!")
        return
    
    user_vector = vectorizer.transform([user_input])  # Преобразуем ввод в вектор
    hotel_vectors = vectorizer.transform(df_grouped["clean_text"])  # Векторы отелей

    similarities = cosine_similarity(user_vector, hotel_vectors).flatten()  # Считаем сходство
    top_indices = similarities.argsort()[::-1][:5]  # Берём топ-5

    result_text.config(state=tk.NORMAL)
    result_text.delete(1.0, tk.END)

    if similarities[top_indices[0]] == 0:
        result_text.insert(tk.END, "❌ По вашему запросу ничего не найдено.")
    else:
        result_text.insert(tk.END, "🎯 Топ 5 рекомендаций:\n\n")
        for idx, i in enumerate(top_indices, 1):
            result_text.insert(
                tk.END, f"{idx}. {df_grouped['name_ru'][i]} (Рейтинг: {df_grouped['rating'][i]:.2f})\n"
            )
    
    result_text.config(state=tk.DISABLED)

# Создание GUI
app = tk.Tk()
app.title("Рекомендации отелей")
app.geometry("500x400")

# Виджеты
tk.Label(app, text="Введите предпочтения:", font=("Arial", 12)).pack(pady=5)
entry = tk.Entry(app, width=50, font=("Arial", 12))
entry.pack(pady=5)

tk.Button(app, text="🔍 Найти отели", font=("Arial", 12), command=recommend_hotels).pack(pady=10)

result_text = scrolledtext.ScrolledText(app, width=60, height=10, font=("Arial", 10), state=tk.DISABLED)
result_text.pack(pady=5)

# Запуск приложения
app.mainloop()
